In [ ]:

# imports and prep
# pip install numpy, gdal, sklearn, tqdm, jupyter
# python -m pip install numpy, gdal, sklearn, tqdm, jupyter

import struct
import numpy as np
import gdal
from sklearn.ensemble import RandomForestRegressor
import random
# For notebook
from tqdm.notebook import tqdm
# # For normal scripts
#from tqdm import tqdm

sample_filename = r"C:\Work\LAGMA\2022.06.09_IVAN\sum_povyd_v2_gbm_gbmover50_r38.img"

channels_filenames = [ r"C:\Work\LAGMA\2022.06.09_IVAN\priv_mod_v10_v2__2t_2010.img",
                       r"C:\Work\LAGMA\2022.06.09_IVAN\ITMAM_SIN_1D250.A2010165_b01fas.img",
                       r"C:\Work\LAGMA\2022.06.09_IVAN\ITMAM_SIN_1D250.A2010165_b02fas.img",
                       r"C:\Work\LAGMA\2022.06.09_IVAN\ITMAM_SIN_1D250.A2010165_b06fas.img" ]

output_filename = r"C:\Work\LAGMA\2022.06.09_IVAN\test_res.img"


In [ ]:
# Get sample size

sample_size = 0

sample_img = gdal.Open( sample_filename, gdal.GA_ReadOnly )

sample_band = sample_img.GetRasterBand( 1 )

size_x = sample_img.RasterXSize
size_y = sample_img.RasterYSize

for i_y in tqdm( range(size_y) ):
    val_1 = sample_band.ReadAsArray(0, i_y, size_x, 1 )
    nonzero_vals = np.count_nonzero( val_1 )
    sample_size += nonzero_vals

sample_img = None
sample_band = None

print(f"SAMPLE SIZE = {sample_size}")


In [ ]:
# Prepare training set

desired_sample_size = 1_000_000
desired_control_size = 1_000_000

sample_x = np.zeros( (desired_sample_size,len(channels_filenames) + 2 ) )
sample_y = np.zeros( (desired_sample_size, ) )

control_x = np.zeros( (desired_sample_size,len(channels_filenames) + 2 ) )
control_y = np.zeros( (desired_sample_size, ) )

sample_img = gdal.Open( sample_filename, gdal.GA_ReadOnly )

channels_imgs = [ gdal.Open( i_filename, gdal.GA_ReadOnly ) for i_filename in channels_filenames ]

size_x = sample_img.RasterXSize
size_y = sample_img.RasterYSize
for i_ch in channels_imgs:
    if size_x != i_ch.RasterXSize or size_y != i_ch.RasterYSize:
        raise SystemExit("sample and channels - different sizes")

sample_band = sample_img.GetRasterBand( 1 )

channels_bands = [ i_ch.GetRasterBand( 1 ) for i_ch in channels_imgs ]

remaining_total_sample_size = sample_size
remaining_desired_sample_size = desired_sample_size
remaining_desired_control_size = desired_control_size
for i_y in tqdm( range(size_y) ):
    sample_vals = sample_band.ReadAsArray(0, i_y, size_x, 1 )
    nonzero_vals = np.nonzero( sample_vals )
    if not nonzero_vals:
        continue

    sample_start = desired_sample_size - remaining_desired_sample_size
    control_start = desired_control_size - remaining_desired_control_size
    sample_index_x = []
    control_index_x = []
    for i_x in nonzero_vals[1]:
        i_rand = random.randint(0,remaining_total_sample_size-1)
        i_rand_c = random.randint(0,remaining_total_sample_size-1)
        if i_rand < remaining_desired_sample_size:
            sample_index_x.append(i_x)
            remaining_desired_sample_size -= 1
        elif i_rand_c < remaining_desired_control_size:
            control_index_x.append(i_x)
            remaining_desired_control_size -= 1
        remaining_total_sample_size -= 1

    if sample_index_x or control_index_x:
        channels_vals = np.zeros( (len(channels_bands), size_x ) )
        for j, i_ch in enumerate( channels_bands ):
            channels_vals[j,:] = i_ch.ReadAsArray(0, i_y, size_x, 1 )
    else:
        continue
    
    if sample_index_x:
        sample_target = np.arange( start = sample_start, stop = sample_start + len(sample_index_x), step = 1 )
        sample_index_x_2d = ( np.zeros( (len(sample_index_x),), dtype=np.int64 ), np.array(sample_index_x) )  
        sample_index_x = np.array(sample_index_x)
        sample_y[sample_target] = sample_vals[sample_index_x_2d]
        sample_x[sample_target,2:] = np.transpose( channels_vals[:,sample_index_x] )
        sample_x[sample_target,0] = sample_index_x
        sample_x[sample_target,1] = np.ones( sample_target.shape )*i_y

    if control_index_x:
        control_target = np.arange( start = control_start, stop = control_start + len(control_index_x), step = 1 )
        control_index_x_2d = ( np.zeros( (len(control_index_x),), dtype=np.int64 ), np.array(control_index_x) )
        control_index_x = np.array(control_index_x)
        control_y[control_target] = sample_vals[control_index_x_2d]
        control_x[control_target,2:] = np.transpose( channels_vals[:,control_index_x] )
        control_x[control_target,0] = control_index_x
        control_x[control_target,1] = np.ones( control_target.shape )*i_y


sample_img = None
for i in range(len(channels_imgs)):
    channels_imgs[i] = None


In [ ]:
# Train classifier

regression = RandomForestRegressor( n_estimators=100, oob_score = True, n_jobs = -1, max_depth=20, verbose = True )

regression.fit( sample_x, sample_y )

print( f"R2 (OOB) = {regression.oob_score_}" )

control_score = regression.score( control_x, control_y )

print( f"R2 (control) = {control_score}" )


In [ ]:
# Apply classifier
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


mask_filename = r"E:\Work\Carbon\carbon_estimates\carbon_2021.02\maps\forest_map\forest_mask\2010_forest_mask.img"

sample_img = gdal.Open( sample_filename, gdal.GA_ReadOnly )

channels_imgs = [ gdal.Open( i_filename, gdal.GA_ReadOnly ) for i_filename in channels_filenames ]

mask_img = gdal.Open( mask_filename, gdal.GA_ReadOnly )

driver = sample_img.GetDriver()
output_img = driver.Create( output_filename, sample_img.RasterXSize,
                            sample_img.RasterYSize, 1, gdal.GDT_UInt16 )
if output_img is None:
    raise SystemExit( f"Failed to create raster {output_filename}")
output_img.SetGeoTransform( sample_img.GetGeoTransform() )
output_img.SetProjection( sample_img.GetProjection() )        
output_img.GetRasterBand(1).Fill( 0 )

size_x = sample_img.RasterXSize
size_y = sample_img.RasterYSize
for i_ch in channels_imgs:
    if size_x != i_ch.RasterXSize or size_y != i_ch.RasterYSize:
        raise SystemExit("sample and channels - different sizes")

if size_x != mask_img.RasterXSize or size_y != mask_img.RasterYSize:
    raise SystemExit("sample and mask - different sizes")

channels_bands = [ i_ch.GetRasterBand( 1 ) for i_ch in channels_imgs ]
output_band = output_img.GetRasterBand( 1 )
mask_band = mask_img.GetRasterBand( 1 )

block_size = 1000
blocks_x = size_x//block_size + 1 if size_x%block_size != 0 else 0
blocks_y = size_y//block_size + 1 if size_y%block_size != 0 else 0

from itertools import product
all_blocks = [ i for i in product( range(blocks_x), range(blocks_y) ) ]
# all_blocks = [ (22,12), (23,12), (24,12), (22,13), (23,13), (24,13), (22,14), (23,14), (24,14) ] # Selection of blocks for quick debug
# all_blocks = [ (22,10) ] # Selection of blocks for quick debug
for i_block in tqdm( all_blocks ):
    start_x = i_block[0]*block_size
    start_y = i_block[1]*block_size
    end_x = min( size_x, start_x + block_size )
    end_y = min( size_y, start_y + block_size )
    to_read_x = end_x - start_x
    to_read_y = end_y - start_y
    i_size = to_read_x*to_read_y

    i_mask = mask_img.ReadAsArray(start_x, start_y, to_read_x, to_read_y )
    mask_indexes = np.nonzero( i_mask )
    y_res = np.zeros( (to_read_y, to_read_x) )
    
    if mask_indexes[0].size != 0:
        x_store = np.zeros( ( len(mask_indexes[0]), len(channels_filenames) + 2 ) )
        for i, i_ch in enumerate(channels_bands):
            i_vals = i_ch.ReadAsArray(start_x, start_y, to_read_x, to_read_y )
            x_store[:,i + 2] = i_vals[mask_indexes]
        x_store[:,0] = mask_indexes[1] + start_x # X
        x_store[:,1] = mask_indexes[0] + start_y # Y

        y_res[mask_indexes] = regression.predict( x_store )
         
    output_band.WriteArray( y_res, start_x, start_y )

sample_img = None
for i in range(len(channels_imgs)):
    channels_imgs[i] = None
output_img = None